# **Quantum Anomaly Detection**

Author: Athan Zhang (athanzxyt) \
Affiliation: Thomas Jefferson High School for Science and Technology, Quantum Research Lab (QLAB)

## Preliminary Setup

We must first set up the working environments of this project.

In [3]:
# Import relevant libraries
import os
import time
import pickle
import covalent as ct
import numpy as np
import torch

# Set a global seed for reproducibility
SEED = 1234

%matplotlib inline

In [ ]:
# Begin the covalent server
os.environ["COVALENT_SERVER_IFACE_ANY"] = "1"
os.system("covalent start --ignore-migrations")
time.sleep(5)                               # Give time for server to start

## Datasets

All of the relevant data to this project can be found in the `./data` of this repository.

In [ ]:
@ct.electron
def load_data_from_pickle(path):
    return pickle.load(open(path, 'rb'))

We will train with mini-batches by processing data using the `torch` `DataLoader` object. However, this object is not pickleable which is a requirement of Covalent electrons. We will build a helper class to fix this.

In [ ]:
from collections.abc import Iterator

class DataGetter:
    """
    A pickleable mock-up of a Python iterator on a torch.utils.Dataloader.
    Provide a dataset X and the resulting object O will allow you to use next(O).
    """

    def __init__(self, X: torch.Tensor, batch_size: int, seed: int = SEED) -> None:
        """ 
        Calls the _init_data method on initialization of a DataGetter Object.
        """
        self.X = X
        self.batch_size = batch_size
        self.data = []
        self._init_data(
            iter(
                torch.utils.data.DataLoader(
                    self.X, batch_size=self.batch_size, shuffle=True)
            )
        )

    def _init_data(self, iterator: Iterator):
        """
        Load the data of the iterator into a list.
        """
        x = next(iterator, None)
        while x is not None:
            self.data.append(x)
            x = next(iterator, None)

    def __next__(self) -> tuple:
        """
        Analogous behaviour to the native Python next() but calling the
        .pop() of the data attribute.
        """
        try:
            return self.data.pop()
        except(IndexError):
            self._init_data(
                iter(
                    torch.utils.data.DataLoader(
                        self.X, batch_size=self.batch_size, shuffle=True)
                )
            )
            return self.data.pop()

Now we create two Covalent electrons to call an instance of the helper class.

In [ ]:
@ct.electron
def get_series_training_cycler(X_tr, n_series_batch):
    X_cycler = DataGetter(X_tr, n_series_batch)
    return X_cycler

@ct.electron
def get_timepoint_training_cycler(X_tr, n_t_batch):
    n_time_points = X_tr.shape[2]
    T  = torch.tensor(np.arange(n_time_points))
    T_cycler = DataGetter(T, n_t_batch)
    return T_cycler

With this, we are able to process data and cyrcle through a training set.